<h1> October Lab Log </h1>

<h3>Conferences/Seminars attended</h3>

<ul>
    <li>NHCS Network Meeting (08/10/2019) --- Nordic High Content Screening</li>
    <li>eSSENCE (15-16/10/2019) --- Swedish E-Science Academy 2019</li>
</ul>

<h3>Cropping Images</h3>

Most of the month was spent on cropping and going through images.

The image set contained 89088 images;

29696 of each channel(d0, d1, d2)

Instructions from Sonja were to extract one object from each image and crop it into a 400x400 image after getting object coordinates from CellProfiler. At first an iterative approach going through and reading every image was used, but this took too much memory and exceeded the memory limit so I take a different approach. Here is a short version with just the d0 channel. 

In [ ]:
#Load packages
import os
import cv2
import glob
import pandas as pd

#Import and process cellPro output data
os.chdir('C:/Users/jxncx/Documents/BINP52/cProfiler/nuclei')
imData = pd.read_csv(r'hoechst-primaryobjectsNuclei.csv')
imData = imData.drop(imData.columns.difference(['ImageNumber', 
                                                        'ObjectNumber',
                                                        'Metadata_FileLocation',
                                                        'Metadata_Plate',
                                                        'Metadata_Well',
                                                        'Location_Center_X',
                                                        'Location_Center_Y']),
                           axis = 1)
imData.to_csv("nucleiObjects.csv", index = False)
#Import and process image(s)
imRaw = glob.glob('testTiffs/*.tiff')
imTest = cv2.imread(imRaw[0])
dim = imTest[0].shape

The directory is changed to wherever the images and data are. The primary objects csv(output of CellProfiler) is read and all columns are dropped except for the ones which are informative such as coordinates, image number, object number, etc. This data is then written to a new csv and we read just one image for now to get the dimensions(here we assume all images have the same dimensions).

In [ ]:
counter = 1
#Iterates through all images(set gets rid of duplicate names)
imgSet = set(imData['Metadata_FileLocation'])
for i in imgSet:
    imName = i.split('/')[-1][:-4]
    imCh = imName[-2:]
    if imCh == 'd0':
        subdir = "ch0Res"
    #Make df have only those not close to the border, just take one image
    nonBordIm = imData.loc[(imData['Metadata_FileLocation'] == i) &
                            (imData['Location_Center_X'] > 200) &
               (imData['Location_Center_X'] < (dim[0] - 200)) &
               (imData['Location_Center_Y'] > 200) &
               (imData['Location_Center_Y'] < (dim[0] - 200))].iloc[0]
    x = int(nonBordIm['Location_Center_X'])
    y = int(nonBordIm['Location_Center_Y'])
    objNum = str(nonBordIm['ObjectNumber'])
    print("READING IMAGE {0} named {1}".format(counter, imName))
    imageVar = cv2.imread('testTiffs/' + imName + '.tiff')
    print("CROPPING IMAGE {0} named {1}".format(counter, imName))
    cropImg = imageVar[y - 200:y + 200, x - 200:x + 200].copy()
    cv2.imwrite("searchCrop/{0}/{1}_{2}_{3}_{4}.png".format(subdir,imName,
                objNum, str(x), str(y)), cropImg)
    print("IMAGE SAVED FOR --- {0} \n PROGRESS : {1}".format(imName,
          ((counter / len(imgSet)) * 100)))
    counter += 1

Here, we initialise a counter at 1.

Then, we use the set function to get rid of duplicate names in the data, since the column has a row with the image name for every object found in that image. Now, we have a list with each image name and no duplicates. Now for a for loop which goes through every image and the information found in the CellProfiler analysis.

First, we check the channel, in this test case we only had d0 images and we have an if statement here that names the variable 'subdir' 'ch0Res' for channel 0 Results. Second, we want to make sure none of these objects are too close to the border of the image or else cropping will result in an image that is not 400x400 but less in at least one of the directions. After checking that it's not too close to the border, it's cropped 200 in all directions, to make 400x400. Slicing is inversed, so the y coordinate goes first, then the x coordinate when cropping an image. This is written to a .png file in the appropriate subdirectory with the image name, object number, and coordinates. 

The print statements are handy for showing progress and this is where the counter we initialised before comes in handy as it tells us what portion of the images are complete.

This new search and crop script was better not only because of memory usage, but also because it finds corresponding images by the name so if two folders don't have the images in the exact same order, it still finds the right corresponding image. This makes it more reliable than a for loop for instance that just goes through two lists.

After running this it was found that none of the channel 1 and 2 images were there. After further inspection, the CellProfiler data file didn't contain any channel 1 or 2 image information. This was because when the first CellProfiler analysis was done those channels were filtered out, so only channel 0 was processed. The CellProfiler pipeline was run again, this time for the other two channels; with LUNARC because a warning was received not to run it on the LUNARC frontend. The resulting primary object files from each channel were used as input for the cropping. 

Upon even further inspection, Salma found the images didn't match. This is because the script goes through the objects in each individual channel that it finds crops that, then moves to the next image. For example, if an image 1 it finds a nucleus in the top left quadrant and that was the first object it found, it would move on to the next image. Then let's say for some image 2 which is just the corresponding image but in a different channel, it couldn't anything, so the first valid object it found was something in the top right quadrant. This poses a problem because these aren't the same cell then. So after talking to Salma, we used the approach of taking the results of channel 0(because it's the easiest to find objects), and simple use the coordinates to crop an image where the channel 0 object would be. If the coordinates are the same and there's a nucleus, we know there's a cell there even if we cannot see it.